In [18]:
import numpy as np
import os

In [19]:
X=np.load('samples.npy')

In [20]:
from tensorflow.keras.layers import Input,LSTM,Dropout,Dense
from tensorflow.keras.models import Model,load_model

In [21]:
model=load_model('autoencodermodel.h5')
model.load_weights('bestweight.hdf5')
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 96)]   0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, None, 1024)   4591616     input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 1024)   0           lstm_4[0][0]                     
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 96)]   0                                            
____________________________________________________________________________________________

In [22]:
#encoder
#input layer
#encoder_input=Input(shape=(None,X.shape[2]))
encoder_input=model.layers[0].output
encoder_output,state_h,state_c=model.layers[4].output
encoder_states=[state_h,state_c]
encoder_model=Model(encoder_input,encoder_states)

In [23]:
#decoder
#define decoder inputs
decoder_input=model.input[1]
decoder_state_input_h=Input(shape=(1024,))
decoder_state_input_c=Input(shape=(1024,))
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]

##Layer 1
decoder_LSTM1=model.layers[5]
decoder_outputs=decoder_LSTM1(decoder_input,initial_state=decoder_states_inputs)

#lstm layer 2
decoder_LSTM2=model.layers[6]
decoder_outputs,state_h,state_c=decoder_LSTM2(decoder_outputs)
decoder_states=[state_h,state_c]

#softmax layer
decoder_dense=model.layers[7]
decoder_outputs=decoder_dense(decoder_outputs)

#define a decoder model
decoder_model=Model([decoder_input]+decoder_states_inputs,
                    [decoder_outputs]+decoder_states)


In [24]:
#random selection starting input vector
start=np.random.randint(0,X.shape[0]-1)
#prediction
state_value=encoder_model.predict(X[start,:,:].reshape(1,X.shape[1],X.shape[2]))
#recursively predict
target_seq=np.zeros((1,1,X.shape[2]))
target_seq[0,0,0]=1
decoded=[]
for i in range(96*16):
    output_tokens,h,c=decoder_model.predict([target_seq]+state_value)
   
    sampled_token_index=np.argmax(output_tokens[0,-1,:])
    decoded.append(sampled_token_index)
    target_seq=np.zeros((1,1,X.shape[2]))
    target_seq[0,0,sampled_token_index]=1
    state_value=[h,c]
print(target_seq)


[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]]]


In [36]:
print(len(decoded))
song=np.empty((10,96,96))
number_of_batch=96*(int(len(decoded)/96))

for j in range(10):
    for i in range(96*j,96*(1+j)):
        k=i%96 
        song[j,k,decoded[i]]=40
       

1536


In [37]:
song.dtype=np.int8

In [38]:
import midi
import pretty_midi
midi_data=midi.samples_to_midi(song,fs=2)
midi_data.write('song1.mid')


In [1]:
import midi